In [ ]:
from typing import List, Optional

import fire

from llama import Dialog, Llama
import torch.distributed as dist
import torch
import os
from data_casual import output_list_test, input_list_test

In [2]:
input_list_test[123]

[{'role': 'system',
  'content': 'Work out your answer step by step and think carefully.'},
 {'role': 'system',
  'content': 'You are a helpful assistant in casual reasoning, is the following reasoning true?'},
 {'role': 'user',
  'content': 'Premise: Suppose there is a closed system of 4 variables, A, B, C and D. All the statistical relations among these 4 variables are as follows: A correlates with B. A correlates with C. B correlates with C. However, A is independent of D. A and D are independent given B. A and D are independent given B and C. A and D are independent given C. B is independent of D. B and D are independent given A. B and D are independent given A and C. B and D are independent given C. C is independent of D. C and D are independent given A. C and D are independent given A and B. C and D are independent given B.\nHypothesis: There exists at least one collider (i.e., common effect) of A and D.'}]

In [3]:
generator = Llama.build("./", "./tokenizer.model",max_seq_len= 1024,max_batch_size= 4)

> initializing model parallel with size 1
> initializing ddp with size 1
> initializing pipeline with size 1


c:\Users\Ribbe\Coding\VSC\FU_Berlin\Actual_Work\llama3\llama\generation.py:95: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(ckpt_path, map_location=

Loaded in 134.55 seconds


In [4]:
Tokenizer = generator.tokenizer
ckpt_dir = "./"
tokenizer_path = "./tokenizer.model"
temperature = 0.6
top_p = 0.9
max_seq_len = 1024
max_batch_size = 4
max_gen_len= None

In [5]:
print(input_list_test[1142])

[{'role': 'system', 'content': 'Work out your answer step by step and think carefully.'}, {'role': 'system', 'content': 'You are a helpful assistant in casual reasoning, is the following reasoning true?'}, {'role': 'user', 'content': 'Premise: Suppose there is a closed system of 6 variables, A, B, C, D, E and F. All the statistical relations among these 6 variables are as follows: A correlates with C. A correlates with D. A correlates with E. A correlates with F. B correlates with D. B correlates with E. B correlates with F. C correlates with D. C correlates with E. C correlates with F. D correlates with E. D correlates with F. E correlates with F. However, A is independent of B. A and B are independent given C. B is independent of C. B and C are independent given A. B and F are independent given A, C, D and E. B and F are independent given A, D and E. C and E are independent given A, B and D. C and E are independent given A, B, D and F. C and F are independent given A, B and D. C and 

In [ ]:
test = generator.chat_completion([input_list_test[1142]], max_gen_len=max_gen_len,top_p=top_p, temperature=temperature)
print(test[0]['generation']['content'])

A delightful puzzle!

Let's break it down step by step.

**Step 1: Analyze the correlations**

From the premises, we know that A correlates with C, D, E, and F. This means that A is associated with all these variables, but we don't know the direction of causality yet.

We also know that B correlates with D, E, and F, but is independent of A. This suggests that B might be a mediator between A and other variables, but we need to be careful not to assume causality.

**Step 2: Examine the independence statements**

The independence statements are crucial here. We know that A is independent of B, which means that A does not directly cause B. This is consistent with our observation that A correlates with C, D, E, and F, but not B.

We also know that B is independent of C, which means that B does not directly cause C. This is consistent with our observation that B correlates with D, E, and F, but not C.

**Step 3: Look for potential confounders**

Given the correlations and independence state

In [ ]:
loss = 0
n = len(input_list_test)

for i in range(n):
    value = generator.chat_completion([input_list_test[i]],
                                       max_gen_len=max_gen_len,
                                       top_p=top_p,
                                       temperature=temperature)
    value2 = 1 if value[0]["generation"]["content"]=="T" else 0
    loss += 1/n*abs(int(output_list_test[i])-int(value2))

print(loss)

1
